In [1]:
from py_env_train import *

# Define the data specifications:
model_data = ["HRES"]
reference_data = ["HSAF"]
task_name = "spatiotemporal"
mm = "MM"  # or DM
date_start="2020-07-01T13"
date_end="2023-04-26T23"
variable = "pr"
mask_type = "no_na"
laginensemble = False

# Define the following for network configs:
loss="mse"
Filters=64
LR=0.001
min_LR=0.000001*LR
BS=16
lr_patience=2
patience=8
lr_factor=0.5
epochs=64
val_split=0.5
n_channels=7
xpixels=128
ypixels=256

######################################################################################################################################################

filename = Func_Train.data_unique_name_generator(model_data, reference_data, task_name, mm, date_start, date_end, variable, mask_type, laginensemble)
data_unique_name=filename[:-4]
training_unique_name = Func_Train.generate_training_unique_name(loss, Filters, LR, min_LR, lr_factor, lr_patience, BS, patience, val_split, epochs)
print(data_unique_name, training_unique_name)

# Create the production data (if doesn't exist)
Func_Train.prepare_produce(PPROJECT_DIR, PRODUCE_FILES, ATMOS_DATA, filename, model_data, reference_data, task_name, mm, date_start, date_end, variable, mask_type, laginensemble)

# load the production data
print("Loading production data...")
produce_files=np.load(PRODUCE_FILES+"/"+"produce_for_"+filename)
produce_x=produce_files["canvas_x"]

# load the model and weights
model = Func_Train.UNET(xpixels, ypixels, n_channels, Filters)
model_path = PSCRATCH_DIR + "/HPT/" + training_unique_name + ".h5"
model.load_weights(model_path)

# produce 
Y_PRED = model.predict(produce_x, verbose=1)
Y_PRED=Y_PRED[..., 0]
Func_Train.de_prepare_produce(Y_PRED, PREDICT_FILES, ATMOS_DATA, filename, 
                              model_data, date_start, date_end, variable, 
                              training_unique_name, onedelay=True)

train_data_hourly_pr_['HRES']_['HSAF']_MM_6__spatiotemporal_128.256_2020-07-01T13_2023-04-26T23_no_na mse_64_0.001_1e-09_0.5_2_16_8_0.5_64
Opening datasets...
Defining X_Produce ...
Data generated
Loading production data...


2023-07-18 22:04:25.893928: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 38246 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:03:00.0, compute capability: 8.0
2023-07-18 22:04:25.895976: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:1 with 38246 MB memory:  -> device: 1, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:44:00.0, compute capability: 8.0
2023-07-18 22:04:25.897746: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:2 with 38246 MB memory:  -> device: 2, name: NVIDIA A100-SXM4-40GB, pci bus id: 0000:84:00.0, compute capability: 8.0
2023-07-18 22:04:25.899392: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1613] Created device /job:localhost/replica:0/task:0/device:GPU:3 with 38246 MB memory:  -> device: 3, name: NVIDIA A100-SXM4-40GB, pci bu

773/773 [==============================] - 42s 38ms/step
